# Project Introduction:
Primer laboratorio de aprendizaje automático, desarrollado por Sergio Barragán Blanco (100472343) y Eduardo Alarcón Navarro (100472175). 
Grupo 17.


# 1. EDA
Existen 22 carácterísticas que definen cada momento, de las cuales ninguna es categórica, todas son numéricas (con la energía suman 23). No existen valores faltantes, pero si los hubiera, los rellenaríamos con la media del valor superior e inferior antes de randomizar el dataset.

No existen tampoco columnas constantes, que se eliminarían. 

Con todo esto, podemos observar que es un problema de regresión.

La variable que estamos intentando predecir es la "energía" que es el valor de la energía generada 24 horas después. 

Por lo tanto, vamos a comenzar con todos los imports necesarios para este proyecto

In [ ]:
import pandas as pd
import numpy as np
import time
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, Normalizer, PowerTransformer
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import train_test_split, GridSearchCV, KFold, RandomizedSearchCV, TimeSeriesSplit, cross_val_score
from sklearn.tree import DecisionTreeRegressor  
from sklearn.pipeline import Pipeline
from sklearn import metrics
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.metrics import mean_squared_error
from sklearn.neural_network import MLPRegressor
from sklearn.svm import SVR



También, vamos a importar el documento con los datos de entrenamiento, y le vamos a eliminar las columnas innecesarias

In [ ]:

# Load the data
data = pd.read_csv('wind_ava.csv.gz', compression='gzip')

# FIlter the data to only include the columns that end in 13
data = data.filter(regex='13$|energy')
#print(data)



Y separar los datos de entrenamiento de los datos de test, con train_test_split

In [ ]:
y = data['energy']
x = data.drop(columns=['energy'])
print(x.head())

# test_size vamos a escoger 0.2 ya que tenemos datos en un rango de 5 años en orden (2015-2019) por lo que (asumiendo que el numero de datos para todos los años es similar) usaremos 2019 como test final.
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, shuffle=False)
#print(f"Fechas de tain {train.iloc[0].datetime}-{train-iloc[-1].datetime}")

# Decidir usando KNN - Scalers

A continuación, usaremos KNN para determinar el método de escalado más apropiado para el problema. 
Dado que no queremos entrenar con datos en el futuro, no podremos usar una validacin cruzada normal (usando Kfold o Shuffle), por lo que tendremos que usar predefined split o Time Series split (el cual es igual a una validación cruzada, pero ignora los posteriores a test), en nuestro caso, usaremos TimeSeriesSplit para todas las validaciones. En cuanto a los scalers, probaremos "StandardScaler", "MinMaxScaler", "RobustScaler", "Normalizer" y "PowerTransformer".


In [ ]:
scalers = [StandardScaler(), MinMaxScaler(), RobustScaler(), Normalizer(), PowerTransformer()]

pipelines = {}
for scaler in scalers:
    pipe = Pipeline([
        ('scaler', scaler),
        ('knn', KNeighborsRegressor())
    ])
    pipelines[str(scaler)[:-2]] = pipe

scores = {}

for name, pipe in pipelines.items():
    scores[name] = -cross_val_score(pipe, X_train, y_train, cv=TimeSeriesSplit(), scoring='neg_root_mean_squared_error').mean()


for name, score in scores.items():
    print (f" {name}: {score}")




# Entrenamiento de diferentes modelos
Lo próximo que haremos, será probar con diferentes métodos de entrenamiento (KNN, árboles de regresión, regresión lineal normal y la variante de Lasso, y SVM. Para elegir el mejor, usaremos la librería time para saber cuánto se tarda en entrenar cada modelo, un dummy para comparar resultados, y el mejor de entre ellos, usando una política de importancia de 20% a tiempo y 80% precisión.

Lo primero que vamos a usar es el dummyRegressor, para tener una representación visual del peor escenario posible y que tanto mejoran nuestros diseños.

Como el scaler que dió mejor scoring fué el PowerTransformer, será el que usaremos de ahora en adelante

In [ ]:
# Create a Dummy regressor to obtain a baseline
scaler = PowerTransformer()

from sklearn.dummy import DummyRegressor
dummy = DummyRegressor(strategy='mean')

pipe = Pipeline([
    ('scaler', scaler),
    ('model', dummy)
])


pipe.fit(X_train, y_train)
y_pred = pipe.predict(X_test)

mse_dummy = metrics.mean_squared_error(y_test, y_pred)
rmse_dummy = np.sqrt(metrics.mean_squared_error(y_test, y_pred))
print(f"MSE: {mse_dummy}")
print(f"RMSE: {rmse_dummy}")



Para empezar, vamos a comparar con un Decision Tree Regresor.

In [ ]:
# DecisionTreeRegressor

param_grid = {
    'criterion': ['friedman_mse', 'absolute_error', 'poisson', 'squared_error'],  # Adjust criterion for regression
    'max_depth': [None, 10, 12, 13, 14, 15, 16, 17],
    'min_samples_split': range(2, 15, 2),
    'min_samples_leaf': range(10, 20, 2),
}

cv = TimeSeriesSplit()
model = RandomizedSearchCV(DecisionTreeRegressor(random_state=7543),  # Use DecisionTreeRegressor
                     param_grid, cv=cv, n_jobs=-1, verbose=1, scoring='neg_root_mean_squared_error', n_iter=50)  # Adjust scoring metric

model2 = GridSearchCV(DecisionTreeRegressor(random_state=7543),  # Use DecisionTreeRegressor
                     param_grid, cv=cv, n_jobs=-1, verbose=1, scoring='neg_root_mean_squared_error')

pipe = Pipeline([
    ('model', model)
])

pipe.fit(X_train, y_train)
y_pred = pipe.predict(X_test)

best_params = pipe.named_steps['model'].best_params_
score = pipe.named_steps['model'].best_score_
print(f"Best params: {best_params}")
print(f"Best score: {-score}")




Como hemos podido observar, el modelo es bastante mejor que el dummy, aproximadamente, un tercio mejor. No obstante probaremos con más modelos para contrastar.

El próximo modelo es la **regresión lineal** normal

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV

# LinearRegression
param_grid = {
    'fit_intercept': [True, False],
    'copy_X': [True, False],
    'n_jobs': [-1],  # -1 indica utilizar todos los núcleos
    'positive': [True, False]
}


# Crear el modelo de regresión lineal
model = LinearRegression()
cv = TimeSeriesSplit()

grid_search_model = GridSearchCV(model,
                           param_grid,
                           cv=cv,
                           scoring="neg_root_mean_squared_error"
                           )


pipe = Pipeline([
    ('scaler', scaler),
    ('model', grid_search_model)
])

# Entrenar el modelo con los datos de entrenamiento
pipe.fit(X_train, y_train)

best_params = pipe.named_steps['model'].best_params_
score = pipe.named_steps['model'].best_score_
print(f"Best params: {best_params}")
print(f"Best score: {-score}")



Como podemos observar, aunque es mejor que el dummy, está bastante peor que el árbol de decisión, no obstante, vamos a probar con  con la variante de Lasso a ver si conseguimos mejorar el entrenamiento:

Con el modelo Lasso, no hemos utilizado un Scaler porque hemos visto que empeoraba el resultado.

In [ ]:
from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error
import numpy as np

# Lasso

# Definir los parámetros que deseas probar
parameters = {
    'alpha': [0.1, 0.5, 1.0, 2.0],  # Valores de regularización
    'fit_intercept': [True, False],
    'copy_X': [True, False], # pendientes de borrar
    'positive': [True, False], 
    'precompute': [True, False], 
    'random_state': [7543], 
    'selection': ['cyclic', 'random'], 
    'tol': [1**(-10), 1**(-5), 1**(-4), 1**(-3), 1**(-2)], 
    'warm_start': [True, False]
}

# Crear el modelo de regresión lineal Lasso
model = Lasso()
cv = TimeSeriesSplit()
grid_search = GridSearchCV(model, 
                           parameters, 
                           scoring='neg_root_mean_squared_error', 
                           cv=cv)

grid_search.fit(X_train, y_train)

# Obtener el mejor modelo y sus hiperparámetros
best_model = grid_search.best_estimator_
best_params = grid_search.best_params_
best_score = -grid_search.best_score_

print("Mejor modelo:",best_model)
print("Mejorres parámetros:",best_params)
print("Score: ", best_score)

Afortunadamente, el modelo es mejor que la regresión lineal normal, pero los árboles le sacan bastante más precisión (aunque tardan más en entrenar, el beneficio sobrepasa los costes.


In [ ]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error
# Usaremos los parametros por default n_slits = 5, max_train_dize = None, test_size = None, gap = 0 
cv = TimeSeriesSplit()

param_grid = {
    'n_neighbors': range(5, 40, 5),
    'weights' : ['uniform', 'distance'],
    'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
    'leaf_size': range(1, 25, 5),
    'p' : [1,2], # distance `= 1 manhatan, p=2 euclidean
}

grid_search = GridSearchCV(KNeighborsRegressor(),
                     param_grid, cv=cv, n_jobs=-1, scoring='neg_root_mean_squared_error')

pipe = Pipeline([
    ('scaler', PowerTransformer()),
    ('model', grid_search)
])


grid_search2 = GridSearchCV(KNeighborsRegressor(),
                     param_grid, cv=cv, n_jobs=-1, scoring='neg_root_mean_squared_error')


pipe.fit(X_train, y_train)
grid_search2.fit(X_train, y_train)


best_model = grid_search2.best_estimator_
best_params = grid_search2.best_params_

best_model_pipe = pipe.named_steps["model"].best_estimator_
best_params_pipe = pipe.named_steps["model"].best_params_
best_score_pipe = -pipe.named_steps["model"].best_score_
best_score = -grid_search2.best_score_

print("Mejor modelo:",best_model, "<----------->", best_model_pipe)
print("Mejorres parámetros:",best_params, "<----------->", best_params_pipe)
print("Score:", best_score , "<----------->", best_score_pipe)


We will now use the SVM to train the model


In [ ]:
cv = TimeSeriesSplit()
model = SVR()

param_grid = {
    'C': range(705, 720),
    'epsilon': [0.1, 0.2, 0.3],
    'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
    'degree': [1, 2, 3, ],
    'gamma': ['scale', 'auto']
}

grid_search = GridSearchCV(model,
                            param_grid,
                            cv=cv, 
                            n_jobs=-1, 
                            scoring='neg_root_mean_squared_error')


pipe = Pipeline([
    ('scaler', PowerTransformer()),
    ('model', grid_search)
])

pipe.fit(X_train, y_train)


print(np.sqrt(mean_squared_error(y_test, y_pred)))
print("best model:", pipe.named_steps['model'].best_estimator_)
print("best params:", pipe.named_steps['model'].best_params_)
print("RMSE: ", -pipe.named_steps['model'].best_score_)

# time taken to run the code:
# time = "13m 46s"
# 384.2528063686719
# best model: SVR(C=719, degree=1, gamma='auto')
# best params: {'C': 719, 'degree': 1, 'epsilon': 0.1, 'gamma': 'auto', 'kernel': 'rbf'}
# RMSE:  380.9578481516544

Y como podemos observar, los mejores modelos son KNN con la pipe creada y el Decision Tree regresor.

Dado que el KNN se entrena en menos tiempo y tiene mejores métricas, será el usado para la versión final

In [ ]:

pipe = Pipeline([
    ('scaler', PowerTransformer()),
    ('model', SVR(C=719, degree=1, epsilon=0.1, gamma='auto', kernel='rbf'))
])


pipe.fit(X_train, y_train)

y_pred_pipe = pipe.predict(X_test)

rmse_pipe = np.sqrt(mean_squared_error(y_test, y_pred_pipe))
print("RMSE:", rmse_pipe)

from joblib import dump, load
dump(pipe, 'best_model_svr.joblib')

Ahora vamos a coger todos los datos que tenemos para entrenar un modelo

In [ ]:
pipe = Pipeline([
    ('scaler', PowerTransformer()),
    ('model', SVR(C=719, degree=1, epsilon=0.1, gamma='auto', kernel='rbf'))
])

pipe.fit(x, y)


## Correlaciones entre parámetros:
Al principio, parece que las columnas lai_lv.13 y lai_hv.13 tienen una correlación, pero según avanza el tiempo, desaparece.

## Escala de los datos
Entre las diferentes columnas de datos, tenemos valores y magnitudes muy dispares.